# Welcome to Full Stack Machine Learning's Week 4 Project!

In the final week, you will return to the workflow you built last week on the [taxi dataset](https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page). 

## Task 1: Deploy the champion
Use what you have learned in the last two weeks to make necessary modifications and to deploy your latest version of the `TaxiFarePrediction` flow to Argo. Use `--branch champion` to denote this deployment as the champion model.

In [32]:
%%writefile ../flows/cloud/taxi_champion.py
from metaflow import FlowSpec, step, card, conda_base, current, Parameter, Flow, trigger, project, S3
from metaflow.cards import Markdown, Table, Image, Artifact

# URL = "https://outerbounds-datasets.s3.us-west-2.amazonaws.com/taxi/latest.parquet"
URL = 's3://outerbounds-datasets/taxi/latest.parquet'
DATETIME_FORMAT = '%Y-%m-%d %H:%M:%S'

# @trigger(events=['s3'])
@conda_base(libraries={'pandas': '1.4.2', 'pyarrow': '11.0.0', 'numpy': '1.21.2', 'scikit-learn': '1.1.2', 'py-xgboost': '1.7.4'})
@project(name="taxi_model")
class TaxiPriceModel(FlowSpec):

    data_url = Parameter("data_url", default=URL)

    def transform_features(self, df):

        # TODO: 
            # Try to complete tasks 2 and 3 with this function doing nothing like it currently is.
            # Understand what is happening.
            # Revisit task 1 and think about what might go in this function.

        obviously_bad_data_filters = [

            df.fare_amount > 0,         # fare_amount in US Dollars
            df.trip_distance <= 100,    # trip_distance in miles
            df.trip_distance > 0,
            df.extra >= 0,
            df.mta_tax >= 0,
            df.tip_amount >= 0,
            df.total_amount > 0,
            df.airport_fee >= 0,

        ]

        for obviously_bad_data_filter in obviously_bad_data_filters:
            df = df[obviously_bad_data_filter]
        df = df.dropna()
        return df

    @step
    def start(self):

        import pandas as pd
        from sklearn.model_selection import train_test_split
        with S3() as s3:
            obj = s3.get(URL)
            df = pd.read_parquet(obj.path)

        self.df = self.transform_features(df)
        
        # self.df = self.transform_features(pd.read_parquet(self.data_url))

        # NOTE: we are split into training and validation set in the validation step which uses cross_val_score.
        # This is a simple/naive way to do this, and is meant to keep this example simple, to focus learning on deploying Metaflow flows.
        # In practice, you want split time series data in more sophisticated ways and run backtests. 
        self.X = self.df["trip_distance"].values.reshape(-1, 1)
        self.y = self.df["total_amount"].values
        self.next(self.linear_model)

    @step
    def linear_model(self):
        "Fit a single variable, linear model to the data."
        from sklearn.linear_model import LinearRegression
        self.model = LinearRegression()

        self.next(self.validate)

    def gather_sibling_flow_run_results(self):

        # storage to populate and feed to a Table in a Metaflow card
        rows = []

        # loop through runs of this flow 
        for run in Flow(self.__class__.__name__):
            if run.id != current.run_id:
                if run.successful:
                    icon = "✅" 
                    msg = "OK"
                    score = str(run.data.scores.mean())
                else:
                    icon = "❌"
                    msg = "Error"
                    score = "NA"
                    for step in run:
                        for task in step:
                            if not task.successful:
                                msg = task.stderr
                row = [Markdown(icon), Artifact(run.id), Artifact(run.created_at.strftime(DATETIME_FORMAT)), Artifact(score), Markdown(msg)]
                rows.append(row)
            else:
                rows.append([Markdown("✅"), Artifact(run.id), Artifact(run.created_at.strftime(DATETIME_FORMAT)), Artifact(str(self.scores.mean())), Markdown("This run...")])
        return rows
                
    
    @card(type="corise")
    @step
    def validate(self):
        from sklearn.model_selection import cross_val_score
        self.scores = cross_val_score(self.model, self.X, self.y, cv=5)
        current.card.append(Markdown("# Taxi Fare Prediction Results"))
        current.card.append(Table(self.gather_sibling_flow_run_results(), headers=["Pass/fail", "Run ID", "Created At", "R^2 score", "Stderr"]))
        self.next(self.end)

    @step
    def end(self):
        print("Success!")


if __name__ == "__main__":
    TaxiPriceModel()

Overwriting ../flows/cloud/taxi_champion.py


In [33]:
! python ../flows/cloud/taxi_champion.py --environment=conda run


Metaflow 2.9.2+ob(v1) executing TaxiPriceModel for user:sandbox
Project: taxi_model, Branch: user.sandbox
Validating your flow...
    The graph looks good!
Running pylint...
    Pylint is happy!
Bootstrapping conda environment...(this could take a few minutes)
2023-05-23 20:05:30.437 Workflow starting (run-id 22), see it in the UI at https://ui-pw-555244755.outerbounds.dev/TaxiPriceModel/22
2023-05-23 20:05:31.366 [22/start/104 (pid 21853)] Task is starting.
2023-05-23 20:05:43.349 [22/start/104 (pid 21853)] Task finished successfully.
2023-05-23 20:05:44.343 [22/linear_model/105 (pid 21967)] Task is starting.
2023-05-23 20:05:46.926 [22/linear_model/105 (pid 21967)] Task finished successfully.
2023-05-23 20:05:47.843 [22/validate/106 (pid 22020)] Task is starting.
2023-05-23 20:05:53.110 [22/validate/106 (pid 22020)] Task finished successfully.
2023-05-23 20:05:53.928 [22/end/107 (pid 22087)] Task is starting.
2023-05-23 20:05:54.965 [22/end/107 (pid 22087)] Success!
2023-05-23 20:05:

In [40]:
! python ../flows/cloud/taxi_champion.py --environment=conda --production --branch taxi_champion --production argo-workflows create


Metaflow 2.9.2+ob(v1) executing TaxiPriceModel for user:sandbox
Project: taxi_model, Branch: prod.taxi_champion
Validating your flow...
    The graph looks good!
Running pylint...
    Pylint is happy!
Deploying taximodel.prod.taxichampion.taxipricemodel to Argo Workflows...
It seems this is the first time you are deploying taximodel.prod.taxichampion.taxipricemodel to Argo Workflows.

A new production token generated.

The namespace of this production flow is
    production:mfprj-k5he6du5acfaci5l-0-hdtv
To analyze results of this production flow add this line in your notebooks:
    namespace("production:mfprj-k5he6du5acfaci5l-0-hdtv")
If you want to authorize other people to deploy new versions of this flow to Argo Workflows, they need to call
    argo-workflows create --authorize mfprj-k5he6du5acfaci5l-0-hdtv
when deploying this flow to Argo Workflows for the first time.
See "Organizing Results" at https://docs.metaflow.org/ for more information about production tokens.

Bootstrapping

## Task 2: Build the challenger
Develop a second model, by using the same `TaxiFarePrediction` architecture. Then, deploy the flow to Argo as the `--branch challenger`. 
<br>
<br>
Hint: Modify the `linear_model` step. 
<br>
Bonus: Write a paragraph summary of how you developed the second model and tested it before deploying the challenger flow. Let us know in Slack what you found challenging about the task? 

In [35]:
%%writefile ../flows/cloud/taxi_challenger.py
from metaflow import FlowSpec, step, card, conda_base, current, Parameter, Flow, trigger, project, S3, retry
from metaflow.cards import Markdown, Table, Image, Artifact

# URL = "https://outerbounds-datasets.s3.us-west-2.amazonaws.com/taxi/latest.parquet"
URL = 's3://outerbounds-datasets/taxi/latest.parquet'
DATETIME_FORMAT = '%Y-%m-%d %H:%M:%S'

# @trigger(events=['s3'])
@conda_base(libraries={'pandas': '1.4.2', 'pyarrow': '11.0.0', 'numpy': '1.21.2', 'scikit-learn': '1.1.2', 'py-xgboost': '1.7.4'})
@project(name="taxi_model")
class TaxiPriceModel(FlowSpec):

    data_url = Parameter("data_url", default=URL)

    def transform_features(self, df):

        obviously_bad_data_filters = [

            df.fare_amount > 0,         # fare_amount in US Dollars
            df.trip_distance <= 100,    # trip_distance in miles
            df.trip_distance > 0,
            df.extra >= 0,
            df.mta_tax >= 0,
            df.tip_amount >= 0,
            df.total_amount > 0,
            df.airport_fee >= 0,

        ]

        for f in obviously_bad_data_filters:
            df = df[f]
        df = df.dropna()
        return df

    @retry(times=2)  # maybe a read error
    @step
    def start(self):

        import pandas as pd
        from sklearn.model_selection import train_test_split

        with S3() as s3:
            obj = s3.get(URL)
            df = pd.read_parquet(obj.path)

        self.df = self.transform_features(df)
        self.X = self.df["trip_distance"].values.reshape(-1, 1)
        self.y = self.df["total_amount"].values
        self.next(self.xgb_model)

    @step
    def xgb_model(self):
        "XGB model"
        from xgboost import XGBRegressor
        self.model = XGBRegressor()
        self.next(self.validate)

    def gather_sibling_flow_run_results(self):

        # storage to populate and feed to a Table in a Metaflow card
        rows = []

        # loop through runs of this flow 
        for run in Flow(self.__class__.__name__):
            if run.id != current.run_id:
                if run.successful:
                    icon = "✅" 
                    msg = "OK"
                    score = str(run.data.scores.mean())
                else:
                    icon = "❌"
                    msg = "Error"
                    score = "NA"
                    for step in run:
                        for task in step:
                            if not task.successful:
                                msg = task.stderr
                row = [Markdown(icon), Artifact(run.id), Artifact(run.created_at.strftime(DATETIME_FORMAT)), Artifact(score), Markdown(msg)]
                rows.append(row)
            else:
                rows.append([Markdown("✅"), Artifact(run.id), Artifact(run.created_at.strftime(DATETIME_FORMAT)), Artifact(str(self.scores.mean())), Markdown("This run...")])
        return rows
                
    
    @card(type="corise")
    @step
    def validate(self):
        from sklearn.model_selection import cross_val_score
        self.scores = cross_val_score(self.model, self.X, self.y, cv=5)
        current.card.append(Markdown("# Taxi Fare Prediction Results"))
        current.card.append(Table(self.gather_sibling_flow_run_results(), headers=["Pass/fail", "Run ID", "Created At", "R^2 score", "Stderr"]))
        self.next(self.end)

    @step
    def end(self):
        print("Success!")


if __name__ == "__main__":
    TaxiPriceModel()

Overwriting ../flows/cloud/taxi_challenger.py


In [36]:
! python ../flows/cloud/taxi_challenger.py --environment=conda run



Metaflow 2.9.2+ob(v1) executing TaxiPriceModel for user:sandbox
Project: taxi_model, Branch: user.sandbox
Validating your flow...
    The graph looks good!
Running pylint...
    Pylint is happy!
Bootstrapping conda environment...(this could take a few minutes)
2023-05-23 20:06:13.001 Workflow starting (run-id 23), see it in the UI at https://ui-pw-555244755.outerbounds.dev/TaxiPriceModel/23
2023-05-23 20:06:14.000 [23/start/109 (pid 22224)] Task is starting.
2023-05-23 20:06:21.219 [23/start/109 (pid 22224)] Task finished successfully.
2023-05-23 20:06:22.100 [23/xgb_model/110 (pid 22315)] Task is starting.
2023-05-23 20:06:25.280 [23/xgb_model/110 (pid 22315)] Task finished successfully.
2023-05-23 20:06:26.416 [23/validate/111 (pid 22367)] Task is starting.
2023-05-23 20:07:43.467 [23/validate/111 (pid 22367)] Task finished successfully.
2023-05-23 20:07:44.334 [23/end/113 (pid 22433)] Task is starting.
2023-05-23 20:07:45.280 [23/end/113 (pid 22433)] Success!
2023-05-23 20:07:46.068

In [41]:
! python ../flows/cloud/taxi_challenger.py --environment=conda --production --branch taxi_challenger argo-workflows create


Metaflow 2.9.2+ob(v1) executing TaxiPriceModel for user:sandbox
Project: taxi_model, Branch: prod.taxi_challenger
Validating your flow...
    The graph looks good!
Running pylint...
    Pylint is happy!
Deploying taximodel.prod.taxichallenger.taxipricemodel to Argo Workflows...
It seems this is the first time you are deploying taximodel.prod.taxichallenger.taxipricemodel to Argo Workflows.

A new production token generated.

The namespace of this production flow is
    production:mfprj-ez7nqne7krwfvxnc-0-iwza
To analyze results of this production flow add this line in your notebooks:
    namespace("production:mfprj-ez7nqne7krwfvxnc-0-iwza")
If you want to authorize other people to deploy new versions of this flow to Argo Workflows, they need to call
    argo-workflows create --authorize mfprj-ez7nqne7krwfvxnc-0-iwza
when deploying this flow to Argo Workflows for the first time.
See "Organizing Results" at https://docs.metaflow.org/ for more information about production tokens.

Bootstr

In [46]:
from metaflow import Flow, namespace
import numpy as np

NAMESPACES = {
    'champion': 'production:mfprj-k5he6du5acfaci5l-0-hdtv',
    'challenger': 'production:mfprj-ez7nqne7krwfvxnc-0-iwza'
}

for k, ns in NAMESPACES.items():
    namespace(ns)
    run = Flow('TaxiPriceModel').latest_successful_run
    score = np.mean(run.data.scores)
    print(f"{k} model accuracy = {100*score:.2f}%")

MetaflowNamespaceMismatch: Object not in namespace 'production:mfprj-k5he6du5acfaci5l-0-hdtv'

## Task 3: Analyze the results
Return to this notebook, and read in the results of the challenger and champion flow using the Metaflow Client API.
<br><br>

#### Questions
- Does your model perform better on the metrics you selected? 
- Think about your day job, how would you go about assessing whether to roll forward the production "champion" to your new model? 
    - What gives you confidence one model is better than another?
    - What kinds of information do you need to monitor to get buy-in from stakeholders that model A is preferable to model B?  

## CONGRATULATIONS! 🎉✨🍾
If you made it this far, you have completed the Full Stack Machine Learning Corise course. 
We are so glad that you chose to learn with us, and hope to see you again in future courses. Stay tuned for more content and come join us in [Slack](http://slack.outerbounds.co/) to keep learning about Metaflow!